# Ekstraksi Informasi dari Berita Kriminal Indonesia menggunakan Named Entity Recognition (NER).

In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [ ]:
ENTITY_TYPES = ["PER", "LOC", "CRIME_TYPE"]
print(f"Skema Entitas yang Digunakan: {ENTITY_TYPES}\n")

Skema Entitas yang Digunakan: ['PER', 'LOC', 'CRIME_TYPE']



In [1]:
keywords = {
    "CRIME_TYPE": [
        "perampokan", "pembunuhan", "pencurian", "penipuan", 
        "pemerkosaan", "penganiayaan", "korupsi", "narkoba", 
        "begal", "curanmor", "penjambretan", "penggelapan"
    ],
    "PER": [
        "tersangka", "pelaku", "korban", "saksi", "polisi", 
        "terdakwa", "terlapor", "pemuda", "pria", "wanita", "oknum"
    ],
    "LOC": [
        "jakarta", "bandung", "surabaya", "medan", "semarang", 
        "yogyakarta", "makassar", "palembang", "denpasar", "batam",
        "jakarta pusat", "jakarta selatan", "jakarta barat", "jakarta timur", "jakarta utara",
        "bandung barat", "jawa barat", "jawa tengah", "jawa timur",
        "rumah sakit", "kantor polisi", "pasar", "jalan raya"
    ]
}

In [2]:
for entity_type, kws in keywords.items():
    print(f"- {entity_type}: {', '.join(kws)}")
print("\n")

- CRIME_TYPE: perampokan, pembunuhan, pencurian, penipuan, pemerkosaan, penganiayaan, korupsi, narkoba, begal, curanmor, penjambretan, penggelapan
- PER: tersangka, pelaku, korban, saksi, polisi, terdakwa, terlapor, pemuda, pria, wanita, oknum
- LOC: jakarta, bandung, surabaya, medan, semarang, yogyakarta, makassar, palembang, denpasar, batam, jakarta pusat, jakarta selatan, jakarta barat, jakarta timur, jakarta utara, bandung barat, jawa barat, jawa tengah, jawa timur, rumah sakit, kantor polisi, pasar, jalan raya




In [3]:
def annotate_tokens_iob(tokenized_lower_words, keywords_dict):
    tags = ['O'] * len(tokenized_lower_words)
    i = 0
    while i < len(tokenized_lower_words):
        matched_this_iteration = False
        
        for entity_type in ENTITY_TYPES:
            entity_keywords_list = keywords_dict.get(entity_type, [])
            
            processed_entity_keywords = []
            for kw_phrase in entity_keywords_list:
                processed_entity_keywords.append(kw_phrase.lower().split()) 
            
            sorted_tokenized_keywords = sorted(processed_entity_keywords, key=len, reverse=True)

            for keyword_tokens in sorted_tokenized_keywords:
                if tokenized_lower_words[i : i + len(keyword_tokens)] == keyword_tokens:
                    tags[i] = f"B-{entity_type}"
                    for j in range(1, len(keyword_tokens)):
                        if i + j < len(tokenized_lower_words):
                            tags[i+j] = f"I-{entity_type}"
                    
                    i += len(keyword_tokens) 
                    matched_this_iteration = True
                    break 
            if matched_this_iteration:
                break 
        
        if not matched_this_iteration:
            i += 1 
            
    return list(zip(tokenized_lower_words, tags))


In [8]:
import pandas as pd
df = pd.read_csv('../data/processed/prepocessed_all_data.csv')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,text_processed,judul_processed,isi_berita_processed,judul_tokens,...,judul_tokens_lower,isi_tokens_lower,isi_berita_tokens,isi_berita_tokens_lower,judul_stemmed,isi_stemmed,judul_stemmed_nostop,isi_stemmed_nostop,judul_pos_nostop,isi_pos_nostop
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...",...,"['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'sangka',...","['laku', 'bunuh', 'dan', 'perkosa', 'aa', '', ...","['4', 'anak', 'bawah', 'umur', 'sangka', 'bunu...","['laku', 'bunuh', 'perkosa', 'aa', '14', 'oran...","[('4', 'NN'), ('anak', 'NN'), ('bawah', 'NN'),...","[('laku', 'NN'), ('bunuh', 'NN'), ('perkosa', ..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,Dalam waktu kurang lebih 3 bulan Polres Probol...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...",...,"['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', 'lama', '3', 'bulan'...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', 'lama', '3', 'bulan'...","['waktu', 'kurang', 'lebih', '3', 'bulan', 'po...","[('26', 'NN'), ('sangka', 'NN'), ('bekuk', 'NN...","[('waktu', 'NN'), ('kurang', 'NN'), ('lebih', ..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Seorang pekerja kafe di Cengkareng MAS 32 didu...,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...",...,"['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","['kerja', 'kafe', 'siram', 'air', 'keras', 'hi...","['orang', 'kerja', 'kafe', 'di', 'cengkareng',...","['kerja', 'kafe', 'siram', 'air', 'keras', 'hi...","['orang', 'kerja', 'kafe', 'cengkareng', 'mas'...","[('kerja', 'NN'), ('kafe', 'NN'), ('siram', 'N...","[('orang', 'NN'), ('kerja', 'NN'), ('kafe', 'N..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hanya gegara rebutan mikrofon untuk bernyanyi ...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...",...,"['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebut', 'mikrofon', 'untu...","['hilang', 'nyawa', 'pria', 'simalungun', 'geg...","['gegara', 'rebut', 'mikrofon', 'nyanyi', 'war...","[('hilang', 'NN'), ('nyawa', 'NN'), ('pria', '...","[('gegara', 'NN'), ('rebut', 'NN'), ('mikrofon..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 J

In [9]:
CSV_FILE_PATH = "../data/processed/prepocessed_all_data.csv"
TEXT_COLUMN_CANDIDATES = ['judul_processed', 'isi_berita_processed']